In [7]:
import pandas as pd
from uszipcode import Zipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

In [8]:
NAMES = {
    0: "loan_id",
    3: "original_interest_rate",
    8: "original_loan_to_value",
    19: "zip_code_short"
}
col_names = [NAMES.get(idx,  "%d"% idx) for idx, _ in enumerate(range(25))]
zipcode = pd.read_csv("zipcode.csv")
zipcode['Zipcode'] = zipcode['Zipcode'].map(lambda x: str(x).zfill(5))
zipcode["3d"] = zipcode['Zipcode'].map(lambda x: x[:3])

In [9]:
Acq_df = pd.read_csv("../FannieMae/2010Q1/Acquisition_2010Q1.txt", header=None, names=col_names,sep="|")

In [10]:
Acq_df['zip_code_short']= Acq_df['zip_code_short'].map(lambda x: str(x).zfill(3))

In [11]:
def get_geo_cent(x):
    res = zipcode[zipcode['3d'] == str(x)][['Lat', 'Long']].mean()
    final = search.by_coordinates(res[0],res[1],returns = 1)
    if len(final) == 0:
        return zipcode[zipcode['3d'] == str(x)].iloc[0,0]
    return final[0].zipcode

In [12]:
Acq_df['zip_geo_center'] = Acq_df['zip_code_short'].map(get_geo_cent)


In [13]:
Acq_df

,loan_id,1,2,original_interest_rate,4,5,6,7,original_loan_to_value,9,...,16,17,18,zip_code_short,20,21,22,23,24,zip_geo_center
0,100010079393,C,"WELLS FARGO BANK, N.A.",4.875,284000,360,01/2010,03/2010,80,80.0,...,1,P,TX,787,NaN,FRM,NaN,NaN,N,78701
1,100013622306,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,180,12/2009,02/2010,63,63.0,...,1,P,CA,932,NaN,FRM,785.0,NaN,N,93219
2,100019943199,R,OTHER,5.000,417000,360,11/2009,01/2010,43,43.0,...,1,S,FL,342,NaN,FRM,808.0,NaN,N,34232
3,100022098429,R,OTHER,5.250,461000,360,01/2010,03/2010,61,61.0,...,2,P,NY,112,NaN,FRM,NaN,NaN,N,11203
4,100023088745,R,"WELLS FARGO BANK, N.A.",5.250,100000,360,11/2009,01/2010,80,80.0,...,1,P,OH,446,NaN,FRM,NaN,NaN,N,44612
5,100027393681,B,"CITIMORTGAGE, INC.",4.250,274000,120,01/2010,03/2010,65,82.0,...,1,P,CA,913,NaN,FRM,NaN,NaN,N,91324
6,100027760956,R,OTHER,5.250,227000,360,01/2010,03/2010,55,55.0,...,1,P,CA,934,NaN,FRM,799.0,NaN,N,93401
7,100031032487,R,"BANK OF AMERICA, N.A.",5.125,730000,360,03/2010,04/2010,63,63.0,...,1,P,MD,208,NaN,FRM,NaN,NaN,N,20850
8,100032715932,R,SUNTRUST MORTGAGE INC.,5.375,96000,360,12/2009,01/2010,75,75.0,...,3,I,LA,711,NaN,FRM,815.0,NaN,N,71103
9,100033339771,R,OTHER,5.250,300000,360,02/2010,04/2010,74,74.0,...,1,P,ID,836,NaN,FRM,NaN,NaN,N,83629


## Part 2 Split trainning and test data

In [11]:
NAMES1 = {0: "loan_id", 1: "DATE", 15: "forclosure_date"}
col_names = [
    NAMES1.get(idx, "%d"% idx) for idx, _ in enumerate(range(31))]
per_df = pd.read_csv("../FannieMae/2010Q1/Performance_2010Q1.txt",header=None,names=col_names,sep="|")

/Users/DanWang/anaconda3/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
split = per_df['DATE'].map(lambda x: int(x[-4:])<2015)

In [14]:
train = per_df[split]
test = per_df[~split]

In [17]:
train

,loan_id,DATE,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,100010079393,02/01/2010,"WELLS FARGO BANK, N.A.",4.875,NaN,0,360,360.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,100010079393,03/01/2010,NaN,4.875,NaN,1,359,358.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,100010079393,04/01/2010,NaN,4.875,NaN,2,358,358.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,100010079393,05/01/2010,NaN,4.875,NaN,3,357,357.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,100010079393,06/01/2010,NaN,4.875,NaN,4,356,355.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
5,100010079393,07/01/2010,NaN,4.875,NaN,5,355,354.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
6,100010079393,08/01/2010,NaN,4.875,281525.61,6,354,353.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
7,100010079393,09/01/2010,NaN,4.875,281166.36,7,353,352.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
8,100010079393,10/01/2010,NaN,4.875,280805.65,8,352,351.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
9,100010079393,11/01/2010,NaN,4.875,280443.47,9,351,350.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [18]:
test

,loan_id,DATE,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
72,100013622306,01/01/2015,NaN,4.750,64120.85,60,120,119.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
73,100013622306,02/01/2015,NaN,4.750,63697.94,61,119,118.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
74,100013622306,03/01/2015,NaN,4.750,63273.36,62,118,117.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
75,100013622306,04/01/2015,NaN,4.750,62847.10,63,117,116.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
76,100013622306,05/01/2015,NaN,4.750,62419.15,64,116,115.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
77,100013622306,06/01/2015,NaN,4.750,61989.51,65,115,114.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
78,100013622306,07/01/2015,NaN,4.750,61558.17,66,114,113.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
79,100013622306,08/01/2015,NaN,4.750,61125.12,67,113,112.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
80,100013622306,09/01/2015,NaN,4.750,60690.35,68,112,111.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
81,100013622306,10/01/2015,NaN,4.750,60253.86,69,111,110.0,01/2025,47300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
